In [2]:
# Install required packages first:
# pip install transformers torch pillow

import torch
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration
from PIL import Image

class SimpleTravelClassifier:
    def __init__(self):
        print("Loading models...")
        
        # Zero-shot text classifier
        self.text_classifier = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli",
            device=0 if torch.cuda.is_available() else -1
        )
        
        # Vision model for image understanding
        self.vision_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.vision_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
        
        # Your 13 features
        self.feature_labels = {
            'attributes_GoodForKids': [
                "family-friendly establishment with children welcome",
                "kid-friendly venue with family activities"
            ],
            'attributes_Ambience_romantic': [
                "romantic atmosphere perfect for couples",
                "intimate dining experience for date nights"
            ],
            'attributes_Ambience_trendy': [
                "trendy modern place with stylish decor",
                "hip contemporary venue"
            ],
            'attributes_Ambience_casual': [
                "casual relaxed atmosphere",
                "informal comfortable setting"
            ],
            'attributes_Ambience_classy': [
                "upscale elegant venue",
                "high-end sophisticated establishment"
            ],
            'attributes_Ambience_intimate': [
                "intimate cozy setting",
                "small quiet venue"
            ],
            'attributes_Ambience_touristy': [
                "popular tourist destination",
                "famous tourist location"
            ],
            'Bars_Night': [
                "bar or nightlife venue",
                "cocktail lounge or pub"
            ],
            'Beauty_Health_Care': [
                "spa wellness center",
                "beauty salon or health facility"
            ],
            'Cafes': [
                "coffee shop or cafe",
                "coffee house with pastries"
            ],
            'GYM': [
                "fitness center or gym",
                "workout facility"
            ],
            'Restaurants_Cuisines': [
                "restaurant serving food",
                "dining establishment"
            ],
            'Shops': [
                "retail store or shop",
                "boutique or shopping establishment"
            ]
        }
        
        print("✅ Ready to classify!")
    
    def classify(self, image_path, text, confidence_threshold=0.5):
        """
        Classify a post with image and text
        
        Args:
            image_path (str): Path to image file
            text (str): Caption and hashtags combined
            confidence_threshold (float): Minimum confidence for positive prediction
            
        Returns:
            dict: Classification results for all 13 features
        """
        
        # Analyze image
        image = Image.open(image_path).convert('RGB')
        inputs = self.vision_processor(image, return_tensors="pt")
        out = self.vision_model.generate(**inputs, max_length=50)
        image_description = self.vision_processor.decode(out[0], skip_special_tokens=True)
        
        # Combine text and image understanding
        full_text = f"{text} {image_description}"
        
        # Classify each feature
        results = {}
        
        for feature, labels in self.feature_labels.items():
            # Add negative labels for better discrimination
            all_labels = labels + ["unrelated venue", "different type of place"]
            
            result = self.text_classifier(full_text, all_labels)
            
            # Get best positive score
            positive_scores = [result['scores'][i] for i, label in enumerate(result['labels']) if label in labels]  
            max_score = max(positive_scores) if positive_scores else 0.0
            
            results[feature] = {
                'prediction': 1 if max_score > confidence_threshold else 0,
                'confidence': round(max_score, 3)
            }
        
        return results

# Usage Example:
if __name__ == "__main__":
    # Initialize classifier
    classifier = SimpleTravelClassifier()
    
    # Classify your image and text
    results = classifier.classify(
        image_path=r"C:\Users\AbdulRahman\Desktop\tst1.jpg",
        text="just a dog"
    )
    
    # Print results
    print("\nClassification Results:")
    print("-" * 40)
    for feature, result in results.items():
        status = "✅ YES" if result['prediction'] == 1 else "❌ NO"
        print(f"{status} {feature}: {result['confidence']}")

Loading models...
✅ Ready to classify!

Classification Results:
----------------------------------------
❌ NO attributes_GoodForKids: 0.064
✅ YES attributes_Ambience_romantic: 0.533
❌ NO attributes_Ambience_trendy: 0.242
✅ YES attributes_Ambience_casual: 0.621
❌ NO attributes_Ambience_classy: 0.249
✅ YES attributes_Ambience_intimate: 0.551
❌ NO attributes_Ambience_touristy: 0.097
❌ NO Bars_Night: 0.354
❌ NO Beauty_Health_Care: 0.027
❌ NO Cafes: 0.051
❌ NO GYM: 0.026
✅ YES Restaurants_Cuisines: 0.855
❌ NO Shops: 0.023


In [8]:
# Install: pip install transformers torch pillow ultralytics

import torch
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from ultralytics import YOLO

class CleanTravelClassifier:
    def __init__(self):
        print("Loading models...")
        
        # Zero-shot text classifier
        self.text_classifier = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli",
            device=0 if torch.cuda.is_available() else -1
        )
        
        # Vision model for image understanding
        self.vision_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.vision_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
        
        # YOLO for object detection
        print("📥 Loading YOLO...")
        self.yolo_model = YOLO('yolov8n.pt')
        
        # Your 13 features
        self.feature_labels = {
            'attributes_GoodForKids': [
                "family-friendly establishment with children welcome, kids playing, families dining together, high chairs visible",
                "kid-friendly venue with family activities, playground equipment, children's menu, toys and games",
                "place where parents bring children, kids running around, family atmosphere with strollers and baby items",
                "venue with families eating together, children laughing, parents with kids, child-friendly environment"
            ],

            'attributes_Ambience_romantic': [
                "romantic atmosphere perfect for couples with candles, dim lighting, intimate seating for two",
                "intimate dining experience for date nights with wine glasses, roses, romantic music",
                "romantic setting with mood lighting, couples holding hands, anniversary dinner atmosphere",
                "date night venue with romantic decor, candlelit tables, couples dining intimately together"
            ],

            'attributes_Ambience_trendy': [
                "trendy modern place with stylish contemporary decor, Instagram-worthy design, hip young crowd",
                "hip contemporary venue with modern furniture, trendy lighting, fashionable clientele",
                "stylish modern establishment with cutting-edge design, trendy atmosphere, social media worthy",
                "fashionable venue with modern aesthetic, trendy decorations, hipster crowd, contemporary style"
            ],

            'attributes_Ambience_casual': [
                "casual relaxed atmosphere with comfortable seating, people in casual clothes, laid-back vibe",
                "informal comfortable setting with relaxed customers, casual dining, everyday atmosphere",
                "laid-back environment with people wearing jeans, casual conversations, comfortable chairs",
                "relaxed venue with informal atmosphere, people eating casually, comfortable and easygoing"
            ],

            'attributes_Ambience_classy': [
                "upscale elegant venue with fine dining, formal atmosphere, sophisticated clientele, expensive decor",
                "high-end sophisticated establishment with luxury furnishing, formal service, elegant design",
                "classy refined venue with expensive wine, formal dining, sophisticated atmosphere, luxury setting",
                "elegant upscale restaurant with fine china, crystal glasses, formal waitstaff, luxury ambiance"
            ],

            'attributes_Ambience_intimate': [
                "romantic atmosphere perfect for couples with candles, dim lighting, intimate seating for two",
                "intimate dining experience for date nights with wine glasses, roses, romantic music",
                "romantic setting with mood lighting, couples holding hands, anniversary dinner atmosphere",
                "date night venue with romantic decor, candlelit tables, couples dining intimately together"
            ],

            'attributes_Ambience_touristy': [
                "popular tourist destination with tour groups, tourists taking photos, guidebooks visible, crowds",
                "famous tourist location with visitors, cameras, tourist buses, international crowd",
                "tourist hotspot with sightseers, vacation atmosphere, tourist attractions, photo opportunities",
                "well-known tourist spot with travelers, vacation photos, tourist signs, famous landmark"
            ],

            'Bars_Night': [
                "bar or nightlife venue with alcohol bottles, bartender mixing drinks, cocktails, beer taps",
                "cocktail lounge or pub with people drinking alcohol, bar stools, liquor display, nighttime crowd",
                "nightlife establishment with drinks being served, people holding beers, bar atmosphere, alcohol service",
                "drinking venue with cocktail glasses, wine bottles, people socializing with drinks, bar counter"
            ],

            'Beauty_Health_Care': [
                "spa wellness center with massage tables, treatment rooms, relaxation area, beauty equipment",
                "beauty salon or health facility with styling chairs, mirrors, beauty treatments, wellness services",
                "health and beauty establishment with spa treatments, facial services, wellness therapy, relaxation",
                "wellness center with massage therapy, beauty treatments, health services, relaxation environment"
            ],

            'Cafes': [
                "coffee shop or cafe with espresso machine, coffee cups, barista, pastries, casual seating",
                "coffee house with pastries, laptop users, coffee beans, casual atmosphere, study environment",
                "cafe establishment with coffee service, people drinking coffee, cafe tables, relaxed atmosphere",
                "coffee place with espresso drinks, people working on laptops, coffee aroma, casual dining"
            ],

            'GYM': [
                "fitness center or gym with exercise equipment, people working out, weights, cardio machines",
                "workout facility with gym equipment, people exercising, fitness machines, athletic activities",
                "fitness gym with people lifting weights, exercise bikes, workout area, athletic equipment",
                "athletic center with fitness training, people exercising, gym machines, workout environment"
            ],

            'Restaurants_Cuisines': [
                "restaurant serving food with dining tables, people eating meals, kitchen, food service, waitstaff",
                "dining establishment with customers eating, food plates, restaurant atmosphere, meal service",
                "food restaurant with people dining, plates of food, restaurant tables, culinary service",
                "eating establishment with food being served, diners enjoying meals, restaurant setting, cuisine"
            ],

            'Shops': [
                "retail store or shop with products for sale, shopping bags, customers browsing, merchandise displays",
                "boutique or shopping establishment with items to buy, retail displays, shopping atmosphere",
                "store with merchandise, people shopping, retail products, commercial environment, sales",
                "shopping venue with products on shelves, customers purchasing, retail atmosphere, store displays"
            ]
        }
        
        print("✅ Ready to classify!")
    
    def get_yolo_description(self, image_path):
        """Generate natural language description from YOLO detections"""
        
        # Run YOLO detection
        results = self.yolo_model(image_path, verbose=False)
        
        detected_objects = []
        
        # Extract high-confidence objects
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    class_name = self.yolo_model.names[class_id]
                    
                    if confidence > 0.5:  # Only confident detections
                        detected_objects.append(class_name)
        
        # Convert objects to natural description
        if not detected_objects:
            return "no specific objects detected"
        
        # Group similar objects
        object_counts = {}
        for obj in detected_objects:
            object_counts[obj] = object_counts.get(obj, 0) + 1
        
        # Create natural language description
        descriptions = []
        for obj, count in object_counts.items():
            if count == 1:
                descriptions.append(f"a {obj}")
            else:
                descriptions.append(f"{count} {obj}s")
        
        # Join naturally
        if len(descriptions) == 1:
            yolo_description = f"contains {descriptions[0]}"
        elif len(descriptions) == 2:
            yolo_description = f"contains {descriptions[0]} and {descriptions[1]}"
        else:
            yolo_description = f"contains {', '.join(descriptions[:-1])}, and {descriptions[-1]}"
        
        return yolo_description
    
    def classify(self, image_path, text, confidence_threshold=0.5):
        """
        Clean classification: BLIP + YOLO descriptions → Text Classifier
        """
        
        print(f"🔍 Analyzing: {text}")
        
        # Step 1: Get BLIP description
        image = Image.open(image_path).convert('RGB')
        inputs = self.vision_processor(image, return_tensors="pt")
        out = self.vision_model.generate(**inputs, max_length=50)
        blip_description = self.vision_processor.decode(out[0], skip_special_tokens=True)
        
        # Step 2: Get YOLO description
        yolo_description = self.get_yolo_description(image_path)
        
        # Step 3: Combine all text sources
        full_text = f"{text}. Scene description: {blip_description}. Objects: {yolo_description}"
        
        print(f"🖼️ BLIP sees: {blip_description}")
        print(f"📦 YOLO sees: {yolo_description}")
        print(f"🔗 Combined: {full_text}")
        
        # Step 4: Classify using enhanced text
        results = {}
        
        for feature, labels in self.feature_labels.items():
            # Add negative labels for better discrimination
            all_labels = labels + ["unrelated venue", "different type of place"]
            
            result = self.text_classifier(full_text, all_labels)
            
            # Get best positive score
            positive_scores = [result['scores'][i] for i, label in enumerate(result['labels']) if label in labels]
            max_score = max(positive_scores) if positive_scores else 0.0
            
            results[feature] = {
                'prediction': 1 if max_score > confidence_threshold else 0,
                'confidence': round(max_score, 3)
            }
        
        return results

# Clean Usage Example:
if __name__ == "__main__":
    # Initialize classifier
    classifier = CleanTravelClassifier()
    
    # Classify your image and text
    results = classifier.classify(
        image_path=r"C:\Users\AbdulRahman\Desktop\tst1.jpg",
        text="having romantic dinner with my fience"
    )
    
    # Print results
    print(f"\n🎯 CLASSIFICATION RESULTS")
    print("-" * 40)
    
    positive_predictions = []
    for feature, result in results.items():
        status = "✅ YES" if result['prediction'] == 1 else "❌ NO"
        print(f"{status} {feature}: {result['confidence']}")
        
        if result['prediction'] == 1:
            positive_predictions.append((feature, result['confidence']))
    
    print(f"\n🏆 DETECTED FEATURES:")
    if positive_predictions:
        for feature, confidence in sorted(positive_predictions, key=lambda x: x[1], reverse=True):
            clean_name = feature.replace('attributes_', '').replace('Ambience_', '')
            print(f"   ✅ {clean_name}: {confidence}")
    else:
        print("   ❌ No features detected")

Loading models...
📥 Loading YOLO...
✅ Ready to classify!
🔍 Analyzing: having romantic dinner with my fience
🖼️ BLIP sees: romantic couple in restaurant
📦 YOLO sees: contains 2 persons and 2 wine glasss
🔗 Combined: having romantic dinner with my fience. Scene description: romantic couple in restaurant. Objects: contains 2 persons and 2 wine glasss

🎯 CLASSIFICATION RESULTS
----------------------------------------
❌ NO attributes_GoodForKids: 0.039
✅ YES attributes_Ambience_romantic: 0.826
❌ NO attributes_Ambience_trendy: 0.092
❌ NO attributes_Ambience_casual: 0.248
✅ YES attributes_Ambience_classy: 0.697
✅ YES attributes_Ambience_intimate: 0.826
❌ NO attributes_Ambience_touristy: 0.034
❌ NO Bars_Night: 0.163
❌ NO Beauty_Health_Care: 0.029
❌ NO Cafes: 0.048
❌ NO GYM: 0.038
✅ YES Restaurants_Cuisines: 0.672
❌ NO Shops: 0.039

🏆 DETECTED FEATURES:
   ✅ romantic: 0.826
   ✅ intimate: 0.826
   ✅ classy: 0.697
   ✅ Restaurants_Cuisines: 0.672


In [ ]:
import json
from PIL import Image
import requests
from io import BytesIO

class TravelClassifierTester:
    def __init__(self, classifier):
        self.classifier = classifier
        self.test_results = []
    
    def run_all_tests(self):
        """Run comprehensive test suite"""
        print("🧪 RUNNING COMPREHENSIVE TEST SUITE")
        print("="*60)
        
        # Test each category
        self.test_restaurants()
        self.test_cafes() 
        self.test_bars()
        self.test_gyms()
        self.test_shops()
        self.test_spas()
        self.test_ambience_detection()
        self.test_family_friendly()
        self.test_edge_cases()
        
        # Generate report
        self.generate_test_report()
    
    def test_case(self, name, image_url, text, expected_features, description=""):
        """Execute single test case"""
        print(f"\n🔍 TEST: {name}")
        print(f"📝 Description: {description}")
        print(f"📸 Image: {image_url}")
        print(f"💬 Text: '{text}'")
        
        # Download image
        try:
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
            image.save(f"test_image_{name.replace(' ', '_').lower()}.jpg")
            image_path = f"test_image_{name.replace(' ', '_').lower()}.jpg"
        except:
            print("❌ Failed to download image")
            return
        
        # Run classification
        results = self.classifier.classify(image_path, text)
        
        # Check results
        passed = 0
        total = len(expected_features)
        
        print(f"\n📊 RESULTS:")
        for feature, expected in expected_features.items():
            actual = results[feature]['prediction']
            confidence = results[feature]['confidence']
            
            if actual == expected:
                status = "✅ PASS"
                passed += 1
            else:
                status = "❌ FAIL"
            
            print(f"  {status} {feature}: Expected {expected}, Got {actual} (conf: {confidence})")
        
        accuracy = passed / total
        print(f"\n🎯 Test Accuracy: {accuracy:.2%} ({passed}/{total})")
        
        # Store results
        self.test_results.append({
            'name': name,
            'accuracy': accuracy,
            'passed': passed,
            'total': total,
            'results': results,
            'expected': expected_features
        })
        
        print("-" * 60)
    
    def test_restaurants(self):
        """Test restaurant classification"""
        print("\n🍽️ TESTING RESTAURANTS")
        
        # Romantic restaurant
        self.test_case(
            name="Romantic Restaurant",
            image_url="https://images.unsplash.com/photo-1514933651103-005eec06c04b?w=500",
            text="Perfect date night dinner! Candlelit atmosphere with amazing wine selection. #romantic #restaurant #datenight #couples #wine",
            expected_features={
                'Restaurants_Cuisines': 1,
                'attributes_Ambience_romantic': 1,
                'attributes_Ambience_intimate': 1,
                'attributes_Ambience_classy': 1,
                'attributes_GoodForKids': 0,
                'Bars_Night': 0,
                'Cafes': 0,
                'GYM': 0
            },
            description="Should detect romantic, upscale restaurant"
        )
        
        # Casual family restaurant
        self.test_case(
            name="Family Restaurant",
            image_url="https://images.unsplash.com/photo-1555396273-367ea4eb4db5?w=500",
            text="Great family dinner! Kids loved the burgers and there's a playground outside. #family #restaurant #kids #burgers #playground",
            expected_features={
                'Restaurants_Cuisines': 1,
                'attributes_GoodForKids': 1,
                'attributes_Ambience_casual': 1,
                'attributes_Ambience_romantic': 0,
                'attributes_Ambience_classy': 0,
                'Bars_Night': 0
            },
            description="Should detect family-friendly casual restaurant"
        )
        
        # Trendy modern restaurant
        self.test_case(
            name="Trendy Restaurant",
            image_url="https://images.unsplash.com/photo-1517248135467-4c7edcad34c4?w=500",
            text="Super hip spot with Instagram-worthy dishes! Modern decor and creative cocktails. #trendy #modern #hipster #instagrammable #cocktails",
            expected_features={
                'Restaurants_Cuisines': 1,
                'attributes_Ambience_trendy': 1,
                'attributes_Ambience_casual': 0,
                'attributes_Ambience_romantic': 0,
                'attributes_GoodForKids': 0
            },
            description="Should detect trendy, modern restaurant"
        )
    
    def test_cafes(self):
        """Test cafe classification"""
        print("\n☕ TESTING CAFES")
        
        # Cozy work cafe
        self.test_case(
            name="Work Cafe",
            image_url="https://images.unsplash.com/photo-1501339847302-ac426a4a7cbb?w=500",
            text="Perfect spot for remote work! Great wifi, amazing coffee, and quiet atmosphere. #cafe #coffee #work #wifi #productivity #latte",
            expected_features={
                'Cafes': 1,
                'attributes_Ambience_casual': 1,
                'attributes_Ambience_intimate': 1,
                'Restaurants_Cuisines': 0,
                'Bars_Night': 0,
                'GYM': 0
            },
            description="Should detect cafe with work-friendly atmosphere"
        )
        
        # Family-friendly cafe
        self.test_case(
            name="Family Cafe",
            image_url="https://images.unsplash.com/photo-1554118811-1e0d58224f24?w=500",
            text="Love bringing the kids here! They have a play area and great hot chocolate. #cafe #family #kids #playground #hotchocolate",
            expected_features={
                'Cafes': 1,
                'attributes_GoodForKids': 1,
                'attributes_Ambience_casual': 1,
                'Restaurants_Cuisines': 0,
                'Bars_Night': 0
            },
            description="Should detect family-friendly cafe"
        )
    
    def test_bars(self):
        """Test bar classification"""
        print("\n🍺 TESTING BARS")
        
        # Cocktail bar
        self.test_case(
            name="Cocktail Bar",
            image_url="https://images.unsplash.com/photo-1514362545857-3bc16c4c7d1b?w=500",
            text="Amazing cocktails and great nightlife! Perfect for drinks with friends. #bar #cocktails #nightlife #drinks #friends #party",
            expected_features={
                'Bars_Night': 1,
                'attributes_Ambience_trendy': 1,
                'attributes_GoodForKids': 0,
                'Restaurants_Cuisines': 0,
                'Cafes': 0,
                'GYM': 0
            },
            description="Should detect nightlife bar"
        )
        
        # Upscale wine bar
        self.test_case(
            name="Wine Bar",
            image_url="https://images.unsplash.com/photo-1572116469696-31de0f17cc34?w=500",
            text="Elegant wine bar with sophisticated atmosphere. Perfect for date nights! #wine #bar #elegant #sophisticated #datenight #romantic",
            expected_features={
                'Bars_Night': 1,
                'attributes_Ambience_classy': 1,
                'attributes_Ambience_romantic': 1,
                'attributes_GoodForKids': 0,
                'attributes_Ambience_casual': 0
            },
            description="Should detect upscale, romantic wine bar"
        )
    
    def test_gyms(self):
        """Test gym classification"""
        print("\n💪 TESTING GYMS")
        
        # Modern fitness center
        self.test_case(
            name="Modern Gym",
            image_url="https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=500",
            text="Great workout today! Modern equipment and clean facilities. #gym #fitness #workout #training #health #exercise",
            expected_features={
                'GYM': 1,
                'attributes_Ambience_trendy': 1,
                'Restaurants_Cuisines': 0,
                'Bars_Night': 0,
                'Cafes': 0,
                'attributes_GoodForKids': 0
            },
            description="Should detect modern gym/fitness center"
        )
        
        # Yoga studio
        self.test_case(
            name="Yoga Studio",
            image_url="https://images.unsplash.com/photo-1544367567-0f2fcb009e0b?w=500",
            text="Peaceful yoga session in this serene studio. Great for mindfulness and relaxation. #yoga #meditation #peaceful #wellness #mindfulness",
            expected_features={
                'GYM': 1,
                'Beauty_Health_Care': 1,
                'attributes_Ambience_intimate': 1,
                'Bars_Night': 0,
                'Restaurants_Cuisines': 0
            },
            description="Should detect yoga/wellness studio"
        )
    
    def test_shops(self):
        """Test retail shop classification"""
        print("\n🛍️ TESTING SHOPS")
        
        # Boutique clothing store
        self.test_case(
            name="Boutique Store",
            image_url="https://images.unsplash.com/photo-1441986300917-64674bd600d8?w=500",
            text="Love this trendy boutique! Amazing selection of unique clothes and accessories. #shopping #boutique #fashion #trendy #clothes",
            expected_features={
                'Shops': 1,
                'attributes_Ambience_trendy': 1,
                'Restaurants_Cuisines': 0,
                'Bars_Night': 0,
                'GYM': 0
            },
            description="Should detect trendy retail boutique"
        )
    
    def test_spas(self):
        """Test spa/beauty classification"""
        print("\n🧖‍♀️ TESTING SPAS & BEAUTY")
        
        # Luxury spa
        self.test_case(
            name="Luxury Spa",
            image_url="https://images.unsplash.com/photo-1571902943202-507ec2618e8f?w=500",
            text="Amazing spa day! Relaxing massage and facial treatments in this peaceful environment. #spa #massage #relaxation #wellness #luxury",
            expected_features={
                'Beauty_Health_Care': 1,
                'attributes_Ambience_classy': 1,
                'attributes_Ambience_intimate': 1,
                'Restaurants_Cuisines': 0,
                'Bars_Night': 0,
                'GYM': 0
            },
            description="Should detect luxury spa/wellness center"
        )
    
    def test_ambience_detection(self):
        """Test ambience attribute detection"""
        print("\n🎭 TESTING AMBIENCE DETECTION")
        
        # Tourist attraction
        self.test_case(
            name="Tourist Restaurant",
            image_url="https://images.unsplash.com/photo-1559329007-40df8a9345d8?w=500",
            text="Famous restaurant that everyone visits! Must-see attraction with great views. #famous #tourist #attraction #mustvisit #views #popular",
            expected_features={
                'Restaurants_Cuisines': 1,
                'attributes_Ambience_touristy': 1,
                'attributes_Ambience_intimate': 0,
                'attributes_Ambience_romantic': 0
            },
            description="Should detect touristy restaurant"
        )
    
    def test_family_friendly(self):
        """Test family-friendly detection"""
        print("\n👨‍👩‍👧‍👦 TESTING FAMILY-FRIENDLY DETECTION")
        
        # Family restaurant with playground
        self.test_case(
            name="Family Restaurant with Play Area",
            image_url="https://images.unsplash.com/photo-1544148103-0773bf10d330?w=500",
            text="Kids had a blast! Restaurant has indoor playground and kids menu. Perfect for families! #family #kids #playground #children #kidfriendly",
            expected_features={
                'Restaurants_Cuisines': 1,
                'attributes_GoodForKids': 1,
                'attributes_Ambience_casual': 1,
                'attributes_Ambience_romantic': 0,
                'Bars_Night': 0
            },
            description="Should strongly detect family-friendly features"
        )
    
    def test_edge_cases(self):
        """Test edge cases and potential failures"""
        print("\n⚠️ TESTING EDGE CASES")
        
        # Ambiguous venue (restaurant + bar)
        self.test_case(
            name="Restaurant Bar Hybrid",
            image_url="https://images.unsplash.com/photo-1578474846511-04ba529f0b88?w=500",
            text="Great dinner and then stayed for drinks! They have both restaurant and bar areas. #restaurant #bar #dinner #drinks #hybrid",
            expected_features={
                'Restaurants_Cuisines': 1,
                'Bars_Night': 1,  # Should detect BOTH
                'Cafes': 0,
                'GYM': 0
            },
            description="Should detect both restaurant and bar (multi-label)"
        )
        
        # Home cooking (should NOT be restaurant)
        self.test_case(
            name="Home Cooking",
            image_url="https://images.unsplash.com/photo-1556909114-4f6e1ba2d201?w=500",
            text="Made dinner at home tonight! Homemade pasta with fresh ingredients. #homecooking #pasta #dinner #cooking #kitchen",
            expected_features={
                'Restaurants_Cuisines': 0,  # Should NOT detect restaurant
                'Bars_Night': 0,
                'Cafes': 0,
                'GYM': 0,
                'Shops': 0
            },
            description="Should NOT classify home cooking as restaurant"
        )
    
    def generate_test_report(self):
        """Generate comprehensive test report"""
        print("\n" + "="*60)
        print("📋 COMPREHENSIVE TEST REPORT")
        print("="*60)
        
        total_tests = len(self.test_results)
        total_passed = sum(r['passed'] for r in self.test_results)
        total_possible = sum(r['total'] for r in self.test_results)
        overall_accuracy = total_passed / total_possible if total_possible > 0 else 0
        
        print(f"\n📊 OVERALL PERFORMANCE:")
        print(f"   Total Test Cases: {total_tests}")
        print(f"   Overall Accuracy: {overall_accuracy:.2%} ({total_passed}/{total_possible})")
        
        # Per-category breakdown
        print(f"\n📈 PER-TEST BREAKDOWN:")
        for result in self.test_results:
            status = "✅ PASS" if result['accuracy'] >= 0.8 else "⚠️ NEEDS IMPROVEMENT" if result['accuracy'] >= 0.6 else "❌ FAIL"
            print(f"   {status} {result['name']}: {result['accuracy']:.2%}")
        
        # Feature performance analysis
        feature_performance = {}
        for result in self.test_results:
            for feature, expected in result['expected'].items():
                if feature not in feature_performance:
                    feature_performance[feature] = {'correct': 0, 'total': 0}
                
                actual = result['results'][feature]['prediction']
                feature_performance[feature]['total'] += 1
                if actual == expected:
                    feature_performance[feature]['correct'] += 1
        
        print(f"\n🎯 FEATURE-SPECIFIC PERFORMANCE:")
        for feature, perf in feature_performance.items():
            accuracy = perf['correct'] / perf['total'] if perf['total'] > 0 else 0
            status = "✅" if accuracy >= 0.8 else "⚠️" if accuracy >= 0.6 else "❌"
            print(f"   {status} {feature}: {accuracy:.2%} ({perf['correct']}/{perf['total']})")
        
        # Recommendations
        print(f"\n💡 RECOMMENDATIONS:")
        low_performing = [f for f, p in feature_performance.items() 
                         if (p['correct'] / p['total'] if p['total'] > 0 else 0) < 0.7]
        
        if low_performing:
            print(f"   🔧 Focus training on: {', '.join(low_performing)}")
        else:
            print(f"   🎉 Model performs well across all features!")
        
        if overall_accuracy >= 0.85:
            print(f"   🚀 Model ready for production deployment!")
        elif overall_accuracy >= 0.70:
            print(f"   📈 Model needs minor improvements before production")
        else:
            print(f"   ⚠️ Model needs significant improvement before deployment")

# Usage:
def test_your_model():
    # Initialize your classifier
    classifier = SimpleTravelClassifier()  # Your working classifier
    
    # Create tester
    tester = TravelClassifierTester(classifier)
    
    # Run all tests
    tester.run_all_tests()

if __name__ == "__main__":
    test_your_model()